# Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv
from typing import Union

In [ ]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

# Functions

In [ ]:
def get_top_500_tokens() -> pd.DataFrame:
    url = "https://api.coingecko.com/api/v3/coins/markets"
    parameters = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 250,
        'page': 1,
        'price_change_percentage': '1h,24h,7d,30d'
    }
    
    response_1 = requests.get(url, params=parameters)
    data_1 = response_1.json()
    
    parameters['page'] = 2
    response_2 = requests.get(url, params=parameters)
    data_2 = response_2.json()
    
    data = data_1 + data_2
    
    df = pd.DataFrame(data)
    
    df = df[['id', 'name', 'current_price', 'market_cap', 'price_change_percentage_24h', 'price_change_percentage_7d_in_currency', 'price_change_percentage_30d_in_currency']]
    
    df.columns = ['ID', 'Name', 'Price', 'Market Cap', '1D %', '7D %', '30D %']
    
    return df

In [ ]:
def get_historical_prices(crypto_id: str, vs_currency: str = 'usd', days: int = 180) -> list:
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": API_KEY
    }

    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily',
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Erreur lors de la récupération des prix pour {crypto_id}: {response.status_code}")
        return None
    
    data = response.json()
    prices = data['prices']
    
    df_prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
    
    return df_prices['price'].tolist()

In [ ]:
def add_180_days_data(df_tokens):
    days_columns = [f'Day_{i+1}' for i in range(181)]
    df_prices = pd.DataFrame(columns=days_columns)
    
    for index, row in df_tokens.iterrows():
        crypto_id = row['ID']
        try:
            prices = get_historical_prices(crypto_id)
            if prices:
                if len(prices) < 180:
                    # Compléter avec None jusqu'à ce que la longueur soit 180
                    prices.extend([None] * (181 - len(prices)))
                # Ajouter les prix dans le DataFrame
                df_prices.loc[index] = prices
            else:
                # Si prices est vide ou None, ajouter une ligne avec 180 None
                df_prices.loc[index] = [None] * 181
        except Exception as e:
            print(f"Erreur lors de la récupération des données pour {crypto_id}: {str(e)}")
            df_prices.loc[index] = [None] * 181
        
        time.sleep(1.5)
    df_final = pd.concat([df_tokens.reset_index(drop=True), df_prices.reset_index(drop=True)], axis=1)

    return df_final

In [ ]:
def fill_missing_prices_with_mean(df):
    """
    Remplace les cellules vides dans les colonnes des 180 jours de prix par la moyenne des prix non nuls.
    :param df: DataFrame à nettoyer.
    :return: DataFrame nettoyé avec les prix manquants remplis.
    """
    # Liste des colonnes représentant les 180 jours de prix (Day_1 à Day_180)
    price_columns = [f'Day_{i+1}' for i in range(180)]
    
    # Pour chaque ligne (chaque cryptomonnaie)
    for index, row in df.iterrows():
        # Extraire les prix de la ligne
        prices = row[price_columns]
        
        # Calculer la moyenne des prix non nuls (en ignorant les None ou NaN)
        mean_price = prices[prices.notna()].mean()
        
        # Remplacer les cellules vides (NaN) par la moyenne des prix non nuls
        df.loc[index, price_columns] = prices.fillna(mean_price)
    
    return df


In [ ]:
def calculate_daily_returns(prices):
    """
    Calcule les rendements quotidiens à partir des prix disponibles.
    Ignore les prix manquants (None ou NaN).
    :param prices: Liste des prix journaliers.
    :return: Liste des rendements quotidiens, en ignorant les valeurs manquantes.
    """
    returns = []
    for i in range(1, len(prices)):
        if prices[i-1] is not None and prices[i] is not None and prices[i-1] != 0:
            daily_return = (prices[i] - prices[i-1]) / prices[i-1]
            returns.append(daily_return)
    return returns

In [ ]:
def calculate_volatility(daily_returns):
    """
    Calcule la volatilité (écart-type) des rendements quotidiens disponibles.
    :param daily_returns: Liste des rendements quotidiens.
    :return: Volatilité (écart-type).
    """
    if daily_returns:
        return np.std(daily_returns)
    return None

In [ ]:
def calculate_risk_reward(expected_return, volatility):
    """
    Calcule le ratio Risk-Reward à partir du rendement attendu et du risque.
    :param expected_return: Rendement attendu (moyenne des rendements).
    :param volatility: volatilité.
    :return: Ratio Risk-Reward.
    """
    if expected_return is not None and volatility is not None and volatility != 0:
        return expected_return / volatility
    return None

In [ ]:
def calculate_dynamic_target(market_cap):
    if market_cap > 100_000_000_000:  # Plus de 100 milliards
        return 1.5  # +50%
    elif market_cap > 10_000_000_000:  # Plus de 10 milliards
        return 2  # x2
    elif market_cap > 1_000_000_000:  # Plus de 1 milliard
        return 3  # x3
    else:
        return 5  # x5 pour les plus petites capitalisations

In [ ]:
def add_risk_reward_column(df):
    """
    Ajoute une colonne 'Risk-Reward' au DataFrame en calculant le ratio pour chaque crypto
    en utilisant uniquement les prix disponibles.
    L'expected return est calculé comme étant le prix actuel multiplié par 5.
    :param df: DataFrame contenant les 500 cryptos et leurs prix sur 180 jours.
    :return: DataFrame avec la colonne 'Risk-Reward' ajoutée à la 6e position.
    """
    risk_reward_values = []
    
    for _, row in df.iterrows():
        # Extraire les prix journaliers (assumant que les colonnes Day_1 à Day_180 existent)
        prices = [row[f'Day_{i+1}'] for i in range(180)]
        
        # Calcul des rendements journaliers
        daily_returns = calculate_daily_returns(prices)
        
        # Calcul du risque (volatilité)
        risk = calculate_volatility(daily_returns)
        
        # Expected Return est le prix actuel x5
        expected_return = row['Price'] * calculate_dynamic_target(row['Market Cap'])
        
        # Calcul du Risk-Reward
        risk_reward = calculate_risk_reward(expected_return, risk)
        
        # Ajouter le résultat au tableau
        risk_reward_values.append(risk_reward)
    
    # Ajouter la colonne 'Risk-Reward' au DataFrame
    df['Risk-Reward'] = risk_reward_values
    cols = df.columns.tolist()
    
    # Placer 'Risk-Reward' à la 6e position
    cols.insert(4, cols.pop(cols.index('Risk-Reward')))
    df = df[cols]
    
    return df

In [ ]:
def clear_rows_with_keywords(df, keywords):
    """
    Supprime les lignes où le 'Name' contient un des mots clés.
    :param df: DataFrame à nettoyer.
    :param keywords: Liste de mots clés à exclure.
    :return: DataFrame nettoyé.
    """
    # Appliquer un filtre pour exclure les lignes où 'Name' contient un des mots clés
    df_cleaned = df[~df['Name'].apply(lambda x: any(keyword in x.lower() for keyword in keywords))]
    
    return df_cleaned

In [ ]:
def add_marketcap_rank(df):
    """
    Ajoute une colonne 'MarketCap Rank' indiquant le classement des tokens en fonction de leur Market Cap.
    :param df: DataFrame contenant la colonne 'Market Cap'.
    :return: DataFrame avec la colonne 'MarketCap Rank' ajoutée.
    """
    # Calculer le rang en fonction de la Market Cap (classement décroissant)
    df['MarketCap Rank'] = df['Market Cap'].rank(ascending=False, method='min').astype(int)
    cols = df.columns.tolist()
    # Placer 'Risk-Reward' à la 6e position
    cols.insert(5, cols.pop(cols.index('MarketCap Rank')))
    df = df[cols]
    return df

In [ ]:
def calculate_momentum_score(df):
    df['Momentum'] = (
        0.5 * df['1D %'] +
        0.3 * df['7D %'] +
        0.2 * df['30D %']
    )
    return df

In [ ]:
def calculate_token_score(df, investment=100, alpha=0.4, beta=0.3, gamma=0.3):
    df = calculate_momentum_score(df)
    
    # Calculer le gain potentiel sur 30 jours
    df['Potential_Gain'] = (df['Price'] * (1 + df['30D %'] / 100) - df['Price']) * (investment / df['Price'])
    
    # Normaliser les valeurs
    df['RR_normalized'] = (df['Risk-Reward'] - df['Risk-Reward'].min()) / (df['Risk-Reward'].max() - df['Risk-Reward'].min())
    df['MarketCap_normalized'] = (df['Market Cap'] - df['Market Cap'].min()) / (df['Market Cap'].max() - df['Market Cap'].min())
    df['Momentum_normalized'] = (df['Momentum'] - df['Momentum'].min()) / (df['Momentum'].max() - df['Momentum'].min())
    df['Potential_Gain_normalized'] = (df['Potential_Gain'] - df['Potential_Gain'].min()) / (df['Potential_Gain'].max() - df['Potential_Gain'].min())
    
    # Calculer le score
    df['Score'] = (
        alpha * df['RR_normalized'] +
        beta * df['MarketCap_normalized'] +
        gamma * df['Momentum_normalized'] +
        0.4 * df['Potential_Gain_normalized']  # Ajouter le gain potentiel au score
    )
    
    return df.sort_values('Score', ascending=False).reset_index(drop=True)

# Code

In [ ]:
df_tokens = get_top_500_tokens()

In [ ]:
df_tokens_with_180_days = add_180_days_data(df_tokens)

In [ ]:
df_tokens_with_180_days = fill_missing_prices_with_mean(df_tokens_with_180_days)

In [ ]:
print(df_tokens_with_180_days.head())

In [ ]:
df_tokens_with_180_days.to_excel('top_500_tokens_with_180_days.xlsx', index=False)

In [ ]:
df_tokens_with_180_days = pd.read_excel('./top_500_tokens_with_180_days.xlsx')

In [ ]:
df_tokens_with_180_days.head()

In [ ]:
df_tokens_with_risk_reward = add_risk_reward_column(df_tokens_with_180_days)

# Sauvegarder dans un fichier Excel si nécessaire
df_tokens_with_risk_reward.to_excel('top_500_tokens_with_risk_reward_180.xlsx', index=False)

In [ ]:
keywords_to_exclude = ['wrapped', 'usd', 'tether', 'usdt', 'bridged', 'staked', 'weth', 'wbtc', 'btc']

# Nettoyer le DataFrame en supprimant les lignes indésirables
df_cleaned = clear_rows_with_keywords(df_tokens_with_risk_reward, keywords_to_exclude)

# Sauvegarder le DataFrame nettoyé dans un nouveau fichier Excel
df_cleaned.to_excel('top_500_tokens_cleaned_180.xlsx', index=False)

print("Fichier nettoyé et sauvegardé sous 'top_500_tokens_cleaned.xlsx'")

In [ ]:
df_ranked_tokens = add_marketcap_rank(df_cleaned)
df_ranked_tokens = calculate_token_score(df_ranked_tokens)

# Sauvegarder le DataFrame classé par score dans un nouveau fichier Excel
df_ranked_tokens.to_excel('top_500_tokens_ranked_by_score_180.xlsx', index=False)

print("Tokens classés par score et sauvegardés dans 'top_500_tokens_new_ranked_by_score.xlsx'")